<a href="https://colab.research.google.com/github/Volt-github24/NgramModel/blob/main/NgramModelFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
""""Obligatoire"""
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
""""Obligatoire"""
import os
os.chdir("/content/drive/")
%cd MyDrive/Collab_Stage_L3/
%ls

/content/drive/MyDrive/Collab_Stage_L3
'Copie de Rapport sur les NLG.gdoc'
 corpus.txt
' Generation de langage lstm and ngram good.ipynb'
'Generation de langage naturel corpus francais avec textgnrnn.ipynb'
 NgramModel.ipynb
 Prise_en_main_Spacy.ipynb
'Rapport d'\''implementation du Modele Ngram pour la generation des phrases, basé sur un corpus francais (conversation dans une banque).gdoc'
"Rapport d'implementation du tutoriel sur les NLG.gdoc"
'Test Ngram corpus francais.ipynb'
 textgenrnn_weights.hdf5


In [7]:
"""Obligatoire"""
import spacy # importation de la librairie
!python -m spacy download fr_core_news_sm # pipeline pour le francais

# Fonction de tokenisation
def return_token(sentence):

  nlp = spacy.load("fr_core_news_sm")
  # Tokeniser la phrase
  doc = nlp(sentence)
  # Retourner le texte de chaque token
  return [X.text for X in doc]

# Extraction du corpus avec ponctuation
# liste = []
with open("corpus.txt", "r", encoding='latin-1') as f:
  for line in f.readlines():
    liste.append(line)

# Tokenisation du corpus
corpus_with_punc = [] # corpus est la liste des tokens
for line in liste:
  result = return_token(line)[:-1]
  corpus_with_punc.append(result)

# Enlever la ponctuation dans le corpus
punc = [':', "!", "^", '.', '`', '~', ';', ',', '?', '...', '_', ' ']
corpus = []
for sentence in corpus_with_punc:
  liste_tokens = []
  for token in sentence:
    if token not in punc :
      liste_tokens.append(token)
  corpus.append(liste_tokens)


2022-08-17 13:40:22.951266: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.3 MB 333 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [8]:
import numpy as np
import math
import random
import itertools
import spacy # importation de la librairie
#!python -m spacy download fr_core_news_sm # pipeline pour le francais

class NgramModel():

    def __init__(self):

      self.word_to_index_unigram = {}
      self.index_to_word_unigram = []
      self.word_to_index_ngram = {}
      self.index_to_word_ngram = []
      self.matrix = np.ones((10, 10), dtype = float)
      self.ngram_size = 10


    # Fonction de tokenisation
    def return_token(self, sentence):
      print("tokenise ...")  
      nlp = spacy.load("fr_core_news_sm")
      # Tokeniser la phrase
      doc = nlp(sentence)
      # Retourner le texte de chaque token
      return [X.text for X in doc]


    # La fonction qui me permet de creer la liste des sequences de mots et le dictionaire  seequence de mot : index
    def create_word_index_ngram(self, corpus, ngram_size = 10): # La fonction prend dorenavant un parametre precisant la taille des grams (initialisee à 3)
      print("create word to index ngram ... ")
      ngram_size = self.ngram_size
      first_chaine = "" # Comme il faut creer une chaine de START de taille ngramsize, je l'initialise
      for _ in range(ngram_size):
        first_chaine = first_chaine + "START "
      first_chaine = first_chaine[:-1] # j'enleve le dernier caractere qui est l'espace, cest la chaine START START START...  qui a l'id 0
      word_to_index = {
      first_chaine:0,
      "END":1 # et END a l'id 1
      }
      index_to_word = [first_chaine, 'END'] # J'initialise la liste des mots

      for speech in corpus: # je parcours chaque phrase de mon corpus
          for i in range(len(speech)): # pour chaque mot de la phrse, non pas mot mais token
              word = "" # variable qui va contenir la sequence des mots, la sequence a la taille de ngramsize
              if i < ngram_size -1 : # Dans cette condition je constitue la sequence, et ici specifiquement elle commence par START
                  word = 'START ' * (ngram_size - i - 1)
                  for k in range(i+1): # Je concatene donc le debut des start la avec les tokens suivants
                      word = word + speech[k] + ' '
              else: # Et ici je constitue les sequences qui ne commencent pas par START, genre ce sont les mots du milieu de la phrase, pas du debut
                  for k in range(i-(ngram_size-1), i+1):
                      word = word + speech[k] + ' '

              word = word[:-1] # j'enleve l'espace de la fin
              if word not in index_to_word: # si la sequence n'est pas encore dans la liste je vais l'ajouter, a la derniere ligne
                  word_to_index[word] = len(index_to_word) # et je l'ajoute aussi dans le dictionnaire avec son id est son emplacement  dans la liste des sequences
                  index_to_word.append(word)
              
      return word_to_index, index_to_word # Je retourne le dictionnaire et la liste...  


    # cette fonction me permet de definir la matrice contenant sequence et mot suivant, les sequences etant de taille ngram_size
    def create_ngram_matrix(self, word_to_index, index_to_word, word_to_index_unigram, corpus, ngram_size = 10):
      print(" create ngram matrix ...")
      ngram_size = self.ngram_size
      V = len(index_to_word) 
      V_unigram = len(word_to_index_unigram)
      matrix = np.zeros((V, V_unigram)) # Je definis ma matrice, qui est de taille (nombre de sequence de mots * nombre de mots unigrams)

      first_chaine = "" # Je fabrique la meme premiere chaine la que je vais ajouter a la colonne 0 de la matrice
      for _ in range(ngram_size):
        first_chaine = first_chaine + "START "
      first_chaine = first_chaine[:-1]

      for sentence in corpus:
          for i in range(len(sentence)): # Je parcours donc chaque token de la liste des tokens de la phrase en question
              if i < ngram_size-1: # Dans cette condition je verifie que le token actuel est parmi les premiers tokens de la phrase, cest a dire pour les cas ou il n'a pas tous les token precedent s pour avoir la sequence complete à ettre dans la matrice
                  taille_start = ""
                  if i == 0: # a chaque fois je vais mettre la chaine START START ... dans la premiere colonne de la matrice
                      matrix[word_to_index[first_chaine], word_to_index_unigram[sentence[i]]] += 1 # et mettre cette case à 1 pour marquer le debut d'une nouvelle phrase
                  if len(sentence) > i+1: # si la phrse est asseez longue, je mets la suite des sequences (ici ce sont les sequences qui commencent par START )
                      # je fabrique la chaine qui commence par START à faire correspondre en ligne dans la matrice puis je concatene avec les premiers mots et je fais la correspondance dans la matrice cad chercher son successeur
                      taille_start = "START " * (ngram_size-i-1) 
                      for j in range(i+1):
                          taille_start = taille_start + sentence[j] + " "
                      matrix[word_to_index[taille_start[:-1]], word_to_index_unigram[sentence[i+1]]] += 1  # j'incremente donc la case correspondant a la sequence et au successeur

              elif i >= (len(sentence)-1): # Il s'agit ici du cas ou je suis au dernier caractere de la phrase
                  # je fabrique la chaine dont je cherche a mettre le successeur à END et la taille de la chaine est en fonction de la taille du ngram
                  taille_chaine = ""
                  for l in range(i-(ngram_size-1), i+1):
                      taille_chaine = taille_chaine + sentence[l] + " "
                  matrix[word_to_index[taille_chaine[:-1]], 1] += 1 # je met la colonne de END à 1
              else: # Il s'agit donc du cas où je suis au milieur de la phrase, quand je tombe sur un mot (ou token), je prends simplement ses precedents, autant qu'il en faut pour faire correspondre a la taille de ngramsize
                  taille_chaine = "" 
                  for l in range(i-(ngram_size-1), i+1): # Je concatene ce mot et ses precedents
                      taille_chaine = taille_chaine + sentence[l] + " "
                  matrix[word_to_index[taille_chaine[:-1]], word_to_index_unigram[sentence[i+1]]] += 1 # Puis j'increment la case correspondant e avec le sucesseur en colonne 
      
      return matrix    # Je retourne ensuite la matrice, toute faite
      
    # Cette fonction permet de constituer la liste des mots (sequence de un seul mot), et le dictionnaire correspondant mot : index
    def create_word_index_unigram(self, corpus):
     
      word_to_index = {
      "START":0,
      "END":1
      }
      
      index_to_word = ['START', 'END']

      for speech in corpus:
          for word in speech: # je parcours chaque phrase  je parcours ses tokens
              if word not in index_to_word: # et je n''ajoute un token que s'il n'est pas deja dans la liste des tokens 
                  word_to_index[word] = len(index_to_word) # et dans le dictionnaire, la valeur pour la cle qui est le mot, est simplement a position de ce mot dans la liste des tokens
                  #print(len(index_to_word))
                  index_to_word.append(word) # ensuite j'ajoute ce token
                  
      return word_to_index, index_to_word # et je retourne le dictionnaire et la liste
    

    # Cette fonction prend en entree l'index d'une sequenc de mot pour retourner l'index de son successeur
    def get_next_word(self, index_previous_ngram, matrix, show_nb_possibilities=False): # il prend l'index du mot et retourne l'index du mot suivant
      
      '''
      index_previous_ngram : int, represent the word index in the word_to_index matrix
      matrix : successor matrix, lines represent the n-gram, columns represent the successor

      1. Check how many successors are possible
      2. Select a random word from the possible words, to avoid repetition
      '''
      
      if (matrix[index_previous_ngram]>0).sum() < 2:
          nb_possible = 1
      elif (matrix[index_previous_ngram]>0).sum() > 9:
          nb_possible = 10
      else:
          nb_possible = (matrix[index_previous_ngram]>0).sum()

      if show_nb_possibilities==True:
          print('NUMBER POSSIBLE ', nb_possible)

      top_indexes = matrix[index_previous_ngram].argsort()[-nb_possible:] # constitue la liste des 10 mots suivants les plus succeptibles (avec le plus d'occurence) , 10 etant le nombre de possibiltes, il varie en trre 1 et 10
      random_index = math.floor(random.random()*nb_possible) # juste pour choisir aleatoirement  l'index du prochin mot parmi toutes les possibilites

      index_next_word = top_indexes[random_index] # on choisi un index aleatoirement dans cet ensemble des 10 possibilites
      
      if matrix[index_previous_ngram][index_next_word] == 0: # Si aucun mot n'est successeur, je renvoie le END
        index_next_word = 1 # C'est l'indice du END

      return index_next_word, matrix[index_previous_ngram][index_next_word]  # retourne l'index du prochain mot, et le nombre d'occurence de ce prochain mot sachant le mot precedent dans le corpus


    #fonction permettant de retrouver le 10gram de debut de phrase a partir d'un 10gram
    def get_begining_sentence(self, index_ngram, matrix):
        print(" get beginning sentence from a part of sentence ...")
        # 1. get the word 10 gram
        sentence = self.index_to_word_ngram(index_ngram)
        if "START" in sentence : # il s'agit d'un 10 gram de debut de phrase
          sentence = sentence.split(" ") # je convertis en liste pour enlever toutes les occurences de START
          while 'START' in sentence:
            sentence.remove('START')
          return ' '.join(sentence) + ' ' # Je reconstitue en chaine pour retourner la chaine bien faite

        else: # ici je recupere le 10 gram tel que le successeur est le premier mot de la sequence
          # 1. get the word 10 gram
          sentence = self.index_to_word_ngram(index_ngram)
          successor = sentence.split(" ")[0]
          index_successor = self.word_to_index_unigram(successor)
          

    # Entrainement du modele (creer la liste des sequences, le dictionnaire des sequences, la matrice sequence successeur)
    def train(self, corpus):

      print(" Model training ...")

      #Get word to index and index to word for unigrams
      self.word_to_index_unigram, self.index_to_word_unigram = self.create_word_index_unigram(corpus) # je contruis le dictionnaire et la liste des tokens du corpus en fonction de la taille de ngram (taille de la demie phrase maxi 10)

      #Get word to index and index to word for ngrams
      self.word_to_index_ngram, self.index_to_word_ngram = self.create_word_index_ngram(corpus, self.ngram_size) # Idem matrice equivalente

      #Get ngram matrix
      self.matrix = self.create_ngram_matrix(self.word_to_index_ngram, self.index_to_word_ngram, self.word_to_index_unigram, corpus, self.ngram_size)

      print(" Model training Success !!! ")

    # Fonction pour generer les phrases
    def generate_sentence(self, previous_text, nb_words_after=100): # Il prend juste la phrase à completer, et un parametre qui permet de specifier le nombre de mots qu'il veut à la suite

      print(" Text generation ... \n") # Design

      #1. tokenisation de la phrase en entree
      used_words = self.return_token(previous_text)
      while ' ' in used_words :
          used_words.remove(' ') # J'enleve dans les espaces dans la phrases à completer

      #2. Recuperons les n derniers mot de la demie phrase si n<10 et 10 sinon
      
      last_ngram = "" # J'initialise la variable qui va contenir la demie phrase (ou alors la sequence de 10 mots de la demir phrase)

      if len(used_words) == 0: # S'il n'entre rien genre une chaine vide, je lui retourne ce message d'erreur
          return "Entrez un texte avec au moins un caractere."

      elif len(used_words) < 10: # Cas ou la demie phrase a moins de 10 caracteres, je vais concatener le ngram avec les START devant

          chaine_start = "START " * (self.ngram_size-len(used_words)) # je constitue le debut de la chaine si jamais la phrase a moins de 10 mots

          for h in range(len(used_words), 0, -1):
            last_ngram = last_ngram + used_words[-h] + " " #  Idem je costitue la sequence ...
          
          last_ngram = chaine_start + last_ngram # je concatene les start avec la phrase entree

      else: 

         for h in range(self.ngram_size, 0, -1):
            last_ngram = last_ngram + used_words[-h] + " " #  Idem je constitue la sequence ...   
      
      last_ngram = last_ngram[:-1] # Pour enlever l'espace de la fin

      result = previous_text + " " # J'initialise la variable qui va contenir la phrase complete a la fin

      list_tuple_last_ngram_used_words_result = []
      try:
          index_last_ngram = self.word_to_index_ngram[last_ngram] # Je recupere donc l'index de ce ngram
          list_tuple_last_ngram_used_words_result.append((last_ngram, used_words, result))
      except KeyError: # cas ou la sequence de mot ne corresponds pas à un début de phrase, je cherche toutes les combinaisons avec ces mots entrés
          all_combinations = itertools.permutations(used_words, len(used_words)) # je fabirque les combinaisons
          for one_ngram in all_combinations:#je parcours chacune de ces combinaisons
              last_ngram = ""
              for i in one_ngram:
                  last_ngram = last_ngram + i + " " # je constitue la sequence avec les mots de la phrase
              last_ngram = last_ngram[:-1] # j'enleve l'espace de la fin
              if len(used_words) < 10: # dans le cas ou le nombre de mots etait moins de 10, je complete
                  chaine_start = "START " * (self.ngram_size-len(used_words)) # je constitue le debut de la chaine si jamais la phrase a moins de 10 mots
                  chaine_start = chaine_start + last_ngram # je concatene les start avec la phrase entree
              if chaine_start in list(self.word_to_index_ngram.keys()):# Si la chaine fait partie d'une sequence dans la matrice constituee, je la conserver
                  used_words = last_ngram.split(" ")
                  result = last_ngram + " "
                  list_tuple_last_ngram_used_words_result.append((chaine_start, used_words, result))
              else: # pour dans ce cas verifier si le last_ngram est une element d'un 10gram pas forcement d'un 10gram qui est un debut de phrase
                  for line in list(self.word_to_index_ngram.keys()):
                      if last_ngram in line:
                          used_words = line.split(" ")
                          used_words = [value for value in used_words if value != 'START' and value !=' ']
                          result = ' '.join(used_words) + " "
                          list_tuple_last_ngram_used_words_result.append((line, used_words, result))
                      

      for last_ngram, used_words, result in list_tuple_last_ngram_used_words_result[:10]:
          
          if 'START' in last_ngram:
            print('oui')
          
          result_copy = result
          used_words_copy = used_words[:]

          list_output = [''] # ici donc le resultat sera une liste, de chaque different resultat obtenu, que ce soit pour chaque ngram ou pour chaque ensemble de resultat generee par chque last_ngram
      
          index_last_ngram = self.word_to_index_ngram[last_ngram] # Je recupere donc l'index de ce ngram

          index_last_ngram_copy = index_last_ngram # je fais une copie de l'index de la demie phrase pour recalculer la suite du texte autant de fois plus bas

          if len(used_words) >= 10 : # Si la demie phrase a plus de 06 mots, je lui retourne un seul resultat

              for i in range(nb_words_after):  # Je boucle au plus le nombre de fois qu'il veut de mots 
                  
                  index_next_word, occurence = self.get_next_word(index_last_ngram, self.matrix) # Je recupere le mot suivant de la derniere sequence

                  if(index_next_word!=1):        # Je verifie que le mot que je veux ajouter n'est pas le mot END car cest la fin de la phrase, si cest le cas, je ne l'ajoute juste pas
                    used_words.append(self.index_to_word_unigram[index_next_word]) 
                    result = result + self.index_to_word_unigram[index_next_word] + " "
                  else:
                      break;    

                  last_ngram = ""
                  for h in range(self.ngram_size, 0, -1): # Pour reconstituer la liste de ngram apres avoir trouvé le successeur du précédent
                    last_ngram = last_ngram + used_words[-h] + " "    
                  index_last_ngram = self.word_to_index_ngram[last_ngram[:-1]]

              list_output.append(result[:-1]) # Je retourne donc son un resultat
          
          else : # je veux generer plusieurs sorties si le nombre de mots est moins de 10 caracteres
              nombre_de_sortie = random.randint(6,10) # cette variable contient un nombre aleatoire de resultat que je retourne dans ce cas

              for _ in range(nombre_de_sortie): # Je boucle donc pour faire plusieurs predictions, pour generer plusieurs resultats comme à la base, le nombre de mot etait petit
                  index_last_ngram = index_last_ngram_copy # Je reinitialise donc l'index a la sequence de debut pour refaire la meme operation
                  result = result_copy # Je reinitialise la variable resultat
                  used_words = used_words_copy[:] # Egalement la liste des tokens de la demie phrase
                  while ' ' in used_words : # J'enleve encore tous les espaces dedans
                      used_words.remove(' ')
                  taille_last_ngram = len(used_words[:])

                  for i in range(nb_words_after):  # Je boucle au plus le nombre de fois qu'il veut de mots 
                  
                      index_next_word, occurence = self.get_next_word(index_last_ngram, self.matrix) # Je recupere le mot suivant de la derniere sequence

                      if(index_next_word!=1):        # Je verifie que le mot que je veux ajouter n'est pas le mot END car cest la fin de la phrase, si cest le cas, je ne l'ajoute juste pas
                        used_words.append(self.index_to_word_unigram[index_next_word])
                        result = result + self.index_to_word_unigram[index_next_word] + " "
                      else:
                          break;    

                      # Je vais modifier la taille des ngrams et reformuler la matrice afin de considerer pour la prochaine etape plus une sequence de mot plus grand (chose qui ameliore le resultat)
                      if taille_last_ngram < 10 : # je m'arrete a 10 car, 10 caracteres precedeents pour trouver le suivant est, on va dire  suffisant!
                          
                          taille_last_ngram = taille_last_ngram + 1 # j'incremente la taille de la sequene de mot que je prends pour la prochaine etape
                          """ mettre a jour la valeur de last_ngram avec les START START, etc """  

                      last_ngram = ""
                      chaine_start = "START " * (self.ngram_size-taille_last_ngram) # je constitue le debut de la chaine si jamais la phrase a moins de 10 mots                  
                      for h in range(taille_last_ngram, 0, -1):
                        last_ngram = last_ngram + used_words[-h] + " " #  Idem je constitue la sequence ...
                      
                      last_ngram = chaine_start + last_ngram # je concatene les start avec la les derniers mots de la phrase

                      index_last_ngram = self.word_to_index_ngram[last_ngram[:-1]]
                  
                  list_output.append(result[:-1]) # J'ajoute un  des resultats sorti
                  list_output.append('\n') # Juste pour armoniser l'affichage à la sortie
                
      return (list_output) # je retourne la liste en supprimant les doubons


In [9]:
ngram_model = NgramModel()
ngram_model.train(corpus)

 Model training ...
create word to index ngram ... 
 create ngram matrix ...
 Model training Success !!! 


In [13]:
print(*ngram_model.generate_sentence("professionnelle"))

 Text generation ... 

tokenise ...
 m' appelle Ikome Emmanuel Ndumbe Je suis en formation professionnelle chez Camtel Yaounde Cameroun en tant que Technicien Système Informatique et Réseau Je suis ouvert à de nouvelles opportunités s' il y a des disponibilités dans cette Entreprise Je serai ouvert au travail
